In [ ]:
from langchain.llms import Ollama
import bs4
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders import WebBaseLoader

In [2]:
llm = Ollama(model="llama2")
embeddings_provider = OllamaEmbeddings()

In [3]:
loader = WebBaseLoader(
    ["https://lilianweng.github.io/posts/2023-06-23-agent/"
     ]
)

docs = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter()
document_chunks = text_splitter.split_documents(docs)

In [5]:
vector_store = DocArrayInMemorySearch.from_documents(document_chunks, embeddings_provider)
retriever = vector_store.as_retriever()
output_parser = StrOutputParser()

In [6]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [7]:
contextualize_q_chain = contextualize_q_prompt | llm | output_parser

In [8]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [9]:
def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_q_chain
    else:
        return input["question"]

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [10]:
rag_chain = (
        RunnablePassthrough.assign(
            context=contextualized_question | retriever | format_docs
        )
        | qa_prompt
        | llm
)

In [11]:
chat_history = []

question = "What is a large language model?"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history})
print(ai_msg)
chat_history.extend([HumanMessage(content=question), AIMessage(content=ai_msg)])

A large language model (LLM) is a type of artificial intelligence (AI) model that is trained on a large corpus of text data to generate language outputs that are coherent and natural-sounding. LLMs are typically trained using deep learning algorithms, such as transformers, and can be used for a variety of natural language processing tasks, such as language translation, text summarization, and language generation.

LLMs have been shown to be highly effective at generating coherent and contextually appropriate text, and have the potential to revolutionize many areas of natural language processing. For example, LLMs could be used to generate chatbot responses that are tailored to the user's input, or to generate automated summaries of long documents.

There are several different types of LLMs, including:

1. Language Translation Models: These models are trained to translate text from one language to another. They can be used to translate text in real-time, or to generate translations of l

In [12]:
second_question = "Can you explain the reasoning behind calling it large?"
second_answer = rag_chain.invoke({"question": second_question, "chat_history": chat_history})
print(second_answer)

AI: Sure! When we refer to a language model as "large," we are typically talking about the size of its training dataset and the complexity of the algorithms used to train it.

Large language models are trained on massive datasets that contain billions or even trillions of words. This allows the model to learn patterns and relationships in language that would be difficult or impossible to capture with smaller datasets. For example, a large language model trained on a dataset of millions of books could generate coherent and contextually appropriate text that is comparable to human-written text.

The size of the training dataset is important because it determines how much information the model can learn from the data. The more data the model has to learn from, the better it can generalize to new situations and generate coherent and natural-sounding text.

In addition to the size of the training dataset, the complexity of the algorithms used to train the model is also important. Deep learn